In [1]:
import os
import pandas as pd
import numpy as np
import IPython.display as ipd
import librosa
import librosa.display
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
audio_dataset_path = '/home/trupti/Desktop/Zense_Proj_ML/archive/Data/genres_original'
metadata = pd.read_csv(r'/home/trupti/Desktop/Zense_Proj_ML/archive/Data/features_30_sec.csv')

In [3]:
metadata.head()

,filename,length,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,...,mfcc16_var,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var,label
0,blues.00000.wav,661794,0.350088,0.088757,0.130228,0.002827,1784.165850,129774.064525,2002.449060,85882.761315,...,52.420910,-1.690215,36.524071,-0.408979,41.597103,-2.303523,55.062923,1.221291,46.936035,blues
1,blues.00001.wav,661794,0.340914,0.094980,0.095948,0.002373,1530.176679,375850.073649,2039.036516,213843.755497,...,55.356403,-0.731125,60.314529,0.295073,48.120598,-0.283518,51.106190,0.531217,45.786282,blues
2,blues.00002.wav,661794,0.363637,0.085275,0.175570,0.002746,1552.811865,156467.643368,1747.702312,76254.192257,...,40.598766,-7.729093,47.639427,-1.816407,52.382141,-3.439720,46.639660,-2.231258,30.573025,blues
3,blues.00003.wav,661794,0.404785,0.093999,0.141093,0.006346,1070.106615,184355.942417,1596.412872,166441.494769,...,44.427753,-3.319597,50.206673,0.636965,37.319130,-0.619121,37.259739,-3.407448,31.949339,blues
4,blues.00004.wav,661794,0.308526,0.087841,0.091529,0.002303,1835.004266,343399.939274,1748.172116,88445.209036,...,86.099236,-5.454034,75.269707,-0.916874,53.613918,-4.404827,62.910812,-11.703234,55.195160,blues


In [4]:
def feature_extractor(file):
    audio, sample_rate = librosa.load(file_name, res_type = 'kaiser_fast')
    mfccs_features = librosa.feature.mfcc(y = audio, sr = sample_rate, n_mfcc = 40)
    mfccs_scaled_features = np.mean(mfccs_features.T, axis = 0)
    return mfccs_scaled_features

In [5]:
#metadata.drop(labels = 552, axis = 0, inplace = True)

In [6]:
from tqdm import tqdm

In [8]:
extracted_features = []
for index_num, row in tqdm(metadata.iterrows()):
    try:
        final_class_labels = row["label"]
        file_name = os.path.join(os.path.abspath(audio_dataset_path), final_class_labels + '/', str(row["filename"]))
        data = feature_extractor(file_name)
        extracted_features.append([data, final_class_labels])
    except Exception as e:
        print(f"Error: {e}")
        continue

552it [00:22, 23.58it/s]/tmp/ipykernel_78500/1946415227.py:2: UserWarning: PySoundFile failed. Trying audioread instead.
  audio, sample_rate = librosa.load(file_name, res_type = 'kaiser_fast')
/home/trupti/Desktop/anaconda3/lib/python3.11/site-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
558it [00:23, 20.87it/s]

Error: 


999it [00:43, 23.02it/s]


In [9]:
extracted_features_df = pd.DataFrame(extracted_features, columns = ['feature', 'class'])
extracted_features_df.head()

,feature,class
0,"[-113.59882, 121.57067, -19.162258, 42.363937,...",blues
1,"[-207.52383, 123.98514, 8.947024, 35.867146, 2...",blues
2,"[-90.757164, 140.44089, -29.084547, 31.686693,...",blues
3,"[-199.57513, 150.08612, 5.663406, 26.855278, 1...",blues
4,"[-160.35417, 126.209496, -35.581394, 22.139254...",blues


In [10]:
extracted_features_df['class'].value_counts()

blues        100
classical    100
country      100
disco        100
hiphop       100
metal        100
pop          100
reggae       100
rock         100
jazz          98
Name: class, dtype: int64

In [11]:
X = np.array(extracted_features_df['feature'].tolist())
y = np.array(extracted_features_df['class'].tolist())

In [12]:
X.shape

(998, 40)

In [13]:
y.shape

(998,)

In [153]:
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Print the shapes of the split datasets
print("Shape of X_train:", X_train.shape)
print("Shape of X_test:", X_test.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of y_test:", y_test.shape)

Shape of X_train: (798, 40)
Shape of X_test: (200, 40)
Shape of y_train: (798,)
Shape of y_test: (200,)


In [154]:
X_train

array([[-7.1623047e+01,  8.1108299e+01,  1.6360794e+01, ...,
        -3.3711376e+00, -6.8292708e+00, -1.3781474e+00],
       [-1.2242892e+02,  1.1149043e+02, -1.3700005e+01, ...,
        -1.2905551e+00,  4.6981841e-01, -2.3442950e+00],
       [-4.2159470e+02,  1.7278941e+02,  2.8069195e+01, ...,
        -1.0533409e+00, -6.0324651e-01, -3.6284623e-01],
       ...,
       [-2.6569849e+02,  8.0639633e+01,  9.7376337e+00, ...,
         2.5353398e+00,  2.6059792e+00, -1.3607571e+00],
       [-1.0084442e+02,  6.4227585e+01,  1.2814461e+01, ...,
         2.2403882e+00,  1.4522368e+00,  6.2251014e-01],
       [-2.3530266e+02,  1.4111952e+02, -3.4681889e+01, ...,
        -6.0326366e+00, -4.1950731e+00,  1.0247830e+00]], dtype=float32)

In [155]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

In [156]:
svm_model = SVC()

In [157]:
svm_model.fit(X_train, y_train)

SVC()

In [158]:
y_pred = svm_model.predict(X_test)

In [159]:
y_pred

array(['reggae', 'pop', 'pop', 'pop', 'reggae', 'hiphop', 'pop', 'jazz',
       'reggae', 'metal', 'pop', 'metal', 'country', 'metal', 'reggae',
       'rock', 'reggae', 'hiphop', 'disco', 'pop', 'jazz', 'metal',
       'disco', 'reggae', 'rock', 'metal', 'classical', 'rock', 'reggae',
       'pop', 'classical', 'pop', 'metal', 'jazz', 'metal', 'metal',
       'classical', 'classical', 'metal', 'jazz', 'pop', 'blues', 'pop',
       'metal', 'classical', 'reggae', 'rock', 'pop', 'metal', 'hiphop',
       'metal', 'classical', 'pop', 'jazz', 'blues', 'reggae', 'jazz',
       'jazz', 'jazz', 'pop', 'metal', 'rock', 'metal', 'classical',
       'pop', 'disco', 'jazz', 'pop', 'blues', 'jazz', 'jazz',
       'classical', 'metal', 'classical', 'blues', 'jazz', 'jazz', 'pop',
       'jazz', 'rock', 'rock', 'jazz', 'rock', 'hiphop', 'metal', 'pop',
       'metal', 'blues', 'pop', 'hiphop', 'hiphop', 'hiphop', 'blues',
       'jazz', 'pop', 'country', 'disco', 'reggae', 'metal', 'blues',
       

In [160]:
accuracy = accuracy_score(y_test, y_pred)

In [161]:
print("Accuracy:", accuracy)

Accuracy: 0.455


**Random Forest**

In [162]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [163]:
rf_classifier = RandomForestClassifier(n_estimators=300, random_state=8)

In [164]:
rf_classifier.fit(X_train, y_train)

RandomForestClassifier(n_estimators=300, random_state=8)

In [165]:
rf_predictions = rf_classifier.predict(X_test)

In [166]:
rf_predictions

array(['pop', 'pop', 'disco', 'disco', 'country', 'jazz', 'pop', 'jazz',
       'reggae', 'metal', 'disco', 'disco', 'pop', 'metal', 'reggae',
       'rock', 'reggae', 'hiphop', 'rock', 'country', 'jazz', 'disco',
       'disco', 'hiphop', 'hiphop', 'blues', 'classical', 'classical',
       'reggae', 'disco', 'jazz', 'disco', 'metal', 'jazz', 'metal',
       'metal', 'classical', 'classical', 'metal', 'jazz', 'pop', 'rock',
       'disco', 'hiphop', 'classical', 'jazz', 'country', 'pop', 'metal',
       'reggae', 'metal', 'jazz', 'pop', 'blues', 'blues', 'reggae',
       'classical', 'jazz', 'jazz', 'pop', 'blues', 'disco', 'metal',
       'classical', 'pop', 'disco', 'jazz', 'pop', 'country', 'classical',
       'country', 'classical', 'metal', 'classical', 'disco', 'reggae',
       'blues', 'pop', 'jazz', 'rock', 'rock', 'jazz', 'blues', 'reggae',
       'rock', 'hiphop', 'metal', 'jazz', 'hiphop', 'hiphop', 'hiphop',
       'hiphop', 'reggae', 'classical', 'pop', 'blues', 'blues', '

In [167]:
rf_accuracy = accuracy_score(y_test, rf_predictions)
print("Random Forest Accuracy:", rf_accuracy)

Random Forest Accuracy: 0.66


**KNN**

In [168]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

In [169]:
knn_classifier = KNeighborsClassifier(n_neighbors=3)

In [170]:
knn_classifier.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=3)

In [171]:
knn_predictions = knn_classifier.predict(X_test)

Exception ignored on calling ctypes callback function: <function _ThreadpoolInfo._find_modules_with_dl_iterate_phdr.<locals>.match_module_callback at 0x7fc583e5f380>
Traceback (most recent call last):
  File "/home/trupti/Desktop/anaconda3/lib/python3.11/site-packages/threadpoolctl.py", line 400, in match_module_callback
    self._make_module_from_path(filepath)
  File "/home/trupti/Desktop/anaconda3/lib/python3.11/site-packages/threadpoolctl.py", line 515, in _make_module_from_path
    module = module_class(filepath, prefix, user_api, internal_api)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/trupti/Desktop/anaconda3/lib/python3.11/site-packages/threadpoolctl.py", line 606, in __init__
    self.version = self.get_version()
                   ^^^^^^^^^^^^^^^^^^
  File "/home/trupti/Desktop/anaconda3/lib/python3.11/site-packages/threadpoolctl.py", line 646, in get_version
    config = get_config().split()
             ^^^^^^^^^^^^^^^^^^
AttributeErro

In [172]:
knn_accuracy = accuracy_score(y_test, knn_predictions)

In [173]:
print("KNN Accuracy:", knn_accuracy)

KNN Accuracy: 0.525


observation : as the number of neighbours increases in while calculating KNN, accuracy decreases.

**Gradient Boosting(XGBoost)**

In [174]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score


In [175]:
#pip install xgboost

In [176]:
xgb_classifier = XGBClassifier(n_estimators=100, learning_rate=0.1, random_state=42)


In [177]:
xgb_classifier.fit(X_train, y_train)

/home/trupti/Desktop/anaconda3/lib/python3.11/site-packages/xgboost/sklearn.py:1438: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  or not (self.classes_ == expected_classes).all()


AttributeError: 'bool' object has no attribute 'all'

In [178]:
y_train

array(['pop', 'country', 'classical', 'rock', 'hiphop', 'rock', 'blues',
       'classical', 'country', 'reggae', 'reggae', 'disco', 'pop', 'jazz',
       'hiphop', 'country', 'rock', 'disco', 'hiphop', 'blues', 'reggae',
       'country', 'jazz', 'pop', 'rock', 'jazz', 'reggae', 'rock',
       'hiphop', 'reggae', 'blues', 'blues', 'country', 'pop',
       'classical', 'disco', 'blues', 'country', 'disco', 'jazz', 'disco',
       'classical', 'hiphop', 'blues', 'blues', 'hiphop', 'blues', 'jazz',
       'jazz', 'disco', 'hiphop', 'rock', 'jazz', 'hiphop', 'blues',
       'classical', 'hiphop', 'country', 'blues', 'rock', 'pop', 'blues',
       'country', 'disco', 'reggae', 'hiphop', 'reggae', 'hiphop',
       'hiphop', 'pop', 'classical', 'reggae', 'country', 'reggae',
       'blues', 'classical', 'metal', 'blues', 'metal', 'metal', 'metal',
       'country', 'country', 'country', 'pop', 'disco', 'hiphop',
       'country', 'reggae', 'disco', 'reggae', 'hiphop', 'disco',
       'hiphop

In [179]:
from sklearn.preprocessing import LabelEncoder

# Create a LabelEncoder instance
label_encoder = LabelEncoder()

# Fit and transform the label encoder on y_train
y_train_encoded = label_encoder.fit_transform(y_train)

# Now, y_train_encoded contains integer labels corresponding to your genre names
print(y_train_encoded)

[7 2 1 9 4 9 0 1 2 8 8 3 7 5 4 2 9 3 4 0 8 2 5 7 9 5 8 9 4 8 0 0 2 7 1 3 0
 2 3 5 3 1 4 0 0 4 0 5 5 3 4 9 5 4 0 1 4 2 0 9 7 0 2 3 8 4 8 4 4 7 1 8 2 8
 0 1 6 0 6 6 6 2 2 2 7 3 4 2 8 3 8 4 3 4 9 3 4 0 1 8 8 6 3 9 6 3 5 9 9 6 5
 5 7 5 4 2 3 8 2 0 1 3 3 7 9 9 4 9 1 5 3 6 4 0 3 7 7 3 5 5 3 1 1 5 9 9 0 9
 1 2 8 0 2 1 3 4 3 1 6 6 1 4 1 0 9 1 7 6 1 4 1 1 0 9 6 0 8 4 2 1 7 2 2 5 8
 1 9 4 1 0 7 1 5 0 6 4 6 8 0 7 4 5 6 6 9 3 0 5 6 9 6 2 5 7 6 5 6 4 0 7 4 5
 5 3 0 0 9 2 2 3 3 9 2 0 1 3 4 9 2 6 6 5 5 5 7 1 6 6 5 5 3 1 2 6 8 6 3 4 0
 1 5 5 3 1 1 4 7 4 4 0 7 0 3 9 9 9 6 8 6 0 0 3 5 0 0 3 1 4 8 5 7 1 4 1 4 4
 9 8 6 1 3 8 8 9 6 3 9 6 2 5 3 6 0 3 7 2 1 9 0 3 6 8 3 4 8 9 2 6 0 5 0 6 2
 8 4 1 0 7 4 1 2 2 4 1 8 5 5 5 1 1 4 7 0 6 9 3 2 0 3 1 9 8 7 2 4 6 3 3 9 1
 1 7 0 3 9 1 4 3 6 3 0 8 7 8 4 5 7 0 1 2 8 6 7 8 1 2 6 5 7 0 1 3 2 6 9 1 1
 6 6 1 3 7 7 7 9 6 5 9 0 9 1 8 1 0 5 0 9 8 5 6 4 2 7 6 2 0 7 4 9 3 2 0 9 3
 4 9 8 9 7 1 8 9 5 9 8 5 9 1 9 1 6 7 7 1 2 9 1 4 4 8 5 7 1 0 1 4 9 2 6 8 7
 0 9 0 4 6 2 3 7 5 6 7 7 

In [180]:
# Create an instance of the XGBClassifier with your desired parameters
xgb_classifier = XGBClassifier(n_estimators=100, learning_rate=0.1, random_state=42)

# Fit the model to your training data with integer labels
xgb_classifier.fit(X_train, y_train_encoded)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              objective='multi:softprob', predictor=None, ...)

In [181]:
y_train

array(['pop', 'country', 'classical', 'rock', 'hiphop', 'rock', 'blues',
       'classical', 'country', 'reggae', 'reggae', 'disco', 'pop', 'jazz',
       'hiphop', 'country', 'rock', 'disco', 'hiphop', 'blues', 'reggae',
       'country', 'jazz', 'pop', 'rock', 'jazz', 'reggae', 'rock',
       'hiphop', 'reggae', 'blues', 'blues', 'country', 'pop',
       'classical', 'disco', 'blues', 'country', 'disco', 'jazz', 'disco',
       'classical', 'hiphop', 'blues', 'blues', 'hiphop', 'blues', 'jazz',
       'jazz', 'disco', 'hiphop', 'rock', 'jazz', 'hiphop', 'blues',
       'classical', 'hiphop', 'country', 'blues', 'rock', 'pop', 'blues',
       'country', 'disco', 'reggae', 'hiphop', 'reggae', 'hiphop',
       'hiphop', 'pop', 'classical', 'reggae', 'country', 'reggae',
       'blues', 'classical', 'metal', 'blues', 'metal', 'metal', 'metal',
       'country', 'country', 'country', 'pop', 'disco', 'hiphop',
       'country', 'reggae', 'disco', 'reggae', 'hiphop', 'disco',
       'hiphop

In [182]:
xgb_predictions = xgb_classifier.predict(X_test)

In [183]:
xgb_accuracy = accuracy_score(y_test, xgb_predictions)
print("XGBoost Accuracy:", xgb_accuracy)

XGBoost Accuracy: 0.0


/home/trupti/Desktop/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:226: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  score = y_true == y_pred


In [184]:
import numpy as np

# Assuming y_true and y_pred are arrays or lists
y_true = np.array(y_true)
y_pred = np.array(y_pred)

# Now perform elementwise comparison and calculate accuracy
score = y_true == y_pred
accuracy = np.mean(score)

print(f"Accuracy: {accuracy}")

NameError: name 'y_true' is not defined

In [185]:
from sklearn.tree import DecisionTreeClassifier
classifier = DecisionTreeClassifier(criterion = 'gini')
classifier.fit(X_train, y_train)

DecisionTreeClassifier()

In [186]:
classifier.predict(X_test)

array(['pop', 'pop', 'disco', 'disco', 'jazz', 'jazz', 'pop', 'jazz',
       'jazz', 'metal', 'disco', 'rock', 'hiphop', 'metal', 'reggae',
       'rock', 'reggae', 'hiphop', 'hiphop', 'disco', 'jazz', 'disco',
       'country', 'hiphop', 'hiphop', 'blues', 'classical', 'disco',
       'reggae', 'disco', 'jazz', 'jazz', 'metal', 'jazz', 'rock',
       'disco', 'classical', 'classical', 'metal', 'jazz', 'hiphop',
       'country', 'jazz', 'hiphop', 'blues', 'disco', 'metal', 'pop',
       'blues', 'country', 'metal', 'jazz', 'hiphop', 'disco', 'blues',
       'reggae', 'disco', 'reggae', 'disco', 'reggae', 'metal', 'rock',
       'metal', 'classical', 'pop', 'disco', 'jazz', 'disco', 'reggae',
       'jazz', 'jazz', 'classical', 'metal', 'classical', 'rock',
       'country', 'blues', 'reggae', 'jazz', 'disco', 'rock', 'country',
       'rock', 'reggae', 'metal', 'country', 'metal', 'country', 'reggae',
       'reggae', 'hiphop', 'hiphop', 'disco', 'classical', 'pop',
       'country', 

In [187]:
classifier.score(X_test, y_test)

0.4

In [188]:
from sklearn import tree
tree.plot_tree(classifier)

[Text(0.3814631740805169, 0.9705882352941176, 'x[0] <= -258.467\ngini = 0.9\nsamples = 798\nvalue = [79, 88, 76, 78, 85, 71, 82, 83, 78, 78]'),
 Text(0.04771371769383698, 0.9117647058823529, 'x[5] <= 9.369\ngini = 0.57\nsamples = 106\nvalue = [10, 66, 6, 1, 1, 18, 0, 1, 3, 0]'),
 Text(0.02982107355864811, 0.8529411764705882, 'x[6] <= 1.443\ngini = 0.294\nsamples = 67\nvalue = [1, 56, 1, 1, 1, 5, 0, 1, 1, 0]'),
 Text(0.015904572564612324, 0.7941176470588235, 'x[10] <= -13.722\ngini = 0.102\nsamples = 57\nvalue = [0, 54, 1, 1, 0, 0, 0, 0, 1, 0]'),
 Text(0.007952286282306162, 0.7352941176470589, 'x[32] <= -0.984\ngini = 0.5\nsamples = 2\nvalue = [0, 0, 0, 1, 0, 0, 0, 0, 1, 0]'),
 Text(0.003976143141153081, 0.6764705882352942, 'gini = 0.0\nsamples = 1\nvalue = [0, 0, 0, 0, 0, 0, 0, 0, 1, 0]'),
 Text(0.011928429423459244, 0.6764705882352942, 'gini = 0.0\nsamples = 1\nvalue = [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]'),
 Text(0.02385685884691849, 0.7352941176470589, 'x[22] <= -9.126\ngini = 0.036\nsamp

In [189]:
pip install graphviz

Note: you may need to restart the kernel to use updated packages.


In [190]:
import graphviz
from sklearn.tree import export_graphviz

In [193]:
from IPython.display import display

In [194]:
from sklearn.naive_bayes import GaussianNB

In [195]:
model = GaussianNB()

In [199]:
model.fit(X_train, y_train_encoded)

GaussianNB()

In [200]:
#model.fit(X_train, y_train)

In [201]:
model.predict(X_test)

array([7, 7, 7, 2, 5, 2, 7, 1, 4, 6, 3, 6, 7, 6, 8, 9, 8, 2, 9, 8, 5, 6,
       2, 5, 4, 4, 1, 2, 4, 2, 1, 7, 6, 5, 6, 6, 1, 1, 6, 5, 7, 9, 7, 4,
       0, 5, 9, 7, 6, 8, 6, 1, 7, 1, 0, 8, 1, 5, 5, 7, 6, 6, 6, 1, 7, 6,
       5, 7, 0, 1, 2, 1, 6, 1, 6, 9, 0, 7, 5, 6, 3, 5, 9, 8, 6, 7, 6, 4,
       8, 4, 4, 2, 3, 5, 7, 2, 9, 8, 9, 3, 0, 8, 2, 6, 0, 7, 4, 9, 6, 6,
       9, 1, 2, 6, 2, 7, 5, 7, 7, 2, 2, 0, 7, 7, 4, 3, 1, 9, 4, 2, 6, 9,
       2, 4, 1, 7, 8, 0, 0, 2, 5, 6, 9, 2, 1, 7, 6, 8, 9, 7, 6, 7, 7, 7,
       0, 6, 6, 0, 8, 6, 8, 7, 0, 6, 7, 6, 0, 8, 2, 1, 2, 6, 6, 9, 9, 7,
       2, 4, 9, 8, 8, 1, 7, 4, 2, 5, 6, 2, 9, 8, 7, 9, 7, 2, 3, 0, 0, 0,
       0, 6])

In [202]:
model.score(X_test, y_test)

/home/trupti/Desktop/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:226: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  score = y_true == y_pred


0.0